# Central Limit Theorem (CLT)

The aim of this notebook is to illustrate the Central Limit Theorem (CLT) through concrete examples.

When you add random numbers from different distributions together and plot the distribution of these sums, you end up with a Gaussian distribution, as dictated by the CLT.
The example also illustrates how widths (and therefore uncertainties) are added in quadrature, as one has to divide the sum by the square root of the number of random numbers that went into the sum in order to get a Gaussian of unit width (when using random numbers of unit width, i.e. RMSE $= \sigma = 1$).


## References:
- **R. Barlow**: page 49 (and page 45 for Uniform distribution)
- **G. Cowan**: page 33
- __[Wikipedia: "Central limit theorem"](http://en.wikipedia.org/wiki/Central_limit_theorem)__

## Author(s), contact(s), and dates:
- Author: Troels C. Petersen (NBI)
- Email:  petersen@nbi.dk
- Date:   10th of November 2025

In [1]:
import numpy as np                                     # Matlab like syntax for linear algebra and functions
import matplotlib.pyplot as plt                        # Plots and figures like you know them from Matlab
from iminuit import Minuit                             # The actual fitting tool, better than scipy's
import sys                                             # Modules to see files and folders in directories
import scienceplots
plt.style.use(['science','notebook','grid'])

Here we set some global variables and the random seed for the random number generator (RNG). This ensures reproducability (the same results every time the notebook is restarted). 

In [2]:
verbose = True                        # Print some numbers or not?
N_verbose = 10                        # If so, how many?
save_plots = False                    # Save the plots produced to file(s)?

r = np.random                         # Random generator
r.seed(42)                            # Set a random seed (but a fixed one - more on that later.)

### Distributions to sample from:

Here we set the parameters for the experiement. We are going to play around with these more.
For now we choose to take __10 uniform numbers__ and consider their sum __1000 times__. 

In [3]:
N_experiments = 1000                  # Number of sums produced
N_uniform     = 10                    # Number of uniform numbers used in sum
N_exponential = 0                     # Number of exponential numbers used in sum
N_cauchy      = 0                     # Number of cauchy numbers used in sum

## Counters to put simulated values into:

We initializing some zero-filled numpy arrays and also a counter to count how many of the produced sums that fall outside some range ($\pm 3\sigma$).

In [4]:
x_uniform = np.zeros((N_uniform, N_experiments))
x_exponential = np.zeros((N_exponential, N_experiments))
x_cauchy = np.zeros((N_cauchy, N_experiments))
x_sum = np.zeros((N_experiments))

N3_sigma = 0        # Counter for the number of produced sums, that fall outside +-3 sigma

Next, we loop over "experiments" to make the code more explicit. This could in principle be avoided (which typically also makes the code faster), but for now we keep it, also to be able to comment on what goes on.

In [5]:
for iexp in range(N_experiments): 

    if ((iexp+1) % 500 == 0): 
        print("At iexp : ", iexp+1)         # Show progress!
    sum_value = 0.0                         # sum_value is the number we are going to add random numbers to!
                                            # According to the CLT, it should be Gaussianly distributed.
 
    # Generating uniform numbers (with mean 0, and RMS of 1) and add the numbers in "sum_value"

    # Generating exponential numbers (with mean 0, and RMS of 1) and add...

    # Generating numbers according to a Cauchy distribution (1 / (1 + x^2)) and add...

    N_total = N_uniform + N_exponential + N_cauchy
    sum_value = sum_value / np.sqrt(N_total)              # Ask yourself, why I divide by sqrt(N)?
    x_sum[iexp] = sum_value

    # Count the number of times, that the sum is not within 3 sigma of a unit Gaussian:
    if not (-3.0 < sum_value < 3.0):
        N3_sigma += 1

At iexp :  500
At iexp :  1000


Now we have created three 2-dimensional arrays of shape (N_pdf, N_experiments), e.g. (10, 1000). We now flatten the arrays to get 1D arrays in order to see their distribution:

In [6]:
x_uniform = x_uniform.flatten()
x_exponential = x_exponential.flatten()
x_cauchy = x_cauchy.flatten()

## Plot the input distributions:

It is always important to visualize out data to see if our code produced the expected results, find outliers and just generally get a better understand of it.

We first define the number of bins and the ranges of the different distributions. Note that it is important to think about this - too many/few bins or poor choice of range may ruin the plot, and one should try to get a "regular" bin widths:

In [7]:
N_bins = 100
x_ranges = [(-2.5, 2.5), (-1.5, 8.5), (-5.0, 5.0)]

We now loop over our three distributions and plot them (if they are not empty). In each subplot we plot histograms of the distributions with the given number of bins and ranges as defined above and with the mean, standard deviation and truncated standard deviation for each distribution. 

## Draw output plots with unit Gaussian for reference:

Having visualized the individual distributions above, we now visualize the sum of the different distributions, `x_sum`. We first plot a histogram of it in the range from `xmin` to `xmax`, and overlay it with a unit Gaussian (scaled to match histogram normalisation).

In [8]:
# Define the Gaussian:
def gauss_pdf(x, N, mu, sigma):
    return N / np.sqrt(2 * np.pi) / sigma * np.exp(-0.5 * (x - mu) ** 2 / sigma ** 2)

## Outside $\pm 3 \sigma$

Below we write how many of the experiments had a result outside the range [-3,3], i.e. beyond $\pm 3 \sigma$:

In [9]:
print(f"Number of experiments beyond 3 sigma / total: {N3_sigma:4d} / {N_experiments:d} = {N3_sigma/N_experiments:6.4f}")

Number of experiments beyond 3 sigma / total:    0 / 1000 = 0.0000


- __Question__: *Do the numbers above make sense? What would you expect it to be, if the distribution was Gaussian? If it was not?*

***

First make sure that you understand what the Central Limit Theorem (CLT) states! Then, acquaint yourself with the program. Make sure that you read through it, as many
of these features will be used onwards. Do you understand why the uniform distribution needs to go from $\pm \sqrt 3$ in order to give a distribution with a width of $1$ (i.e. unit) and why you subtract $1$ from the exponential distribution (and how this works at all)?

# Questions:

1. What is the mean and RMSE (i.e. Standard Deviation) of the input distributions?

2. Why is there a $\frac{1}{\sqrt N}$ at the bottom of cell [7] in the line `sum_value = sum_value / np.sqrt(Ntotal)` (when summing up the various contributions to sum)?
    Hint: Assume that I always wanted to compare the distribution of sums with a UNIT Gaussian.

3. Using a sum of 10 uniform random numbers with mean 0 and width 1, what is the expected 
    width of the resulting distribution according to CLT? What is the probability of
    obtaining a number beyond 3 sigma, i.e. how many numbers did you get beyond 3 sigma?
    What would you expect from a true Gaussian distribution?
    And what about the same question for 3.5 sigma? And 4.0 sigma?
    Put additional counters into the code, to test any effects in the tails (which have little statistics as it is).
    Increase the number of experiments run to (much) more than 1000...

4. Now try to add 10 exponential (i.e. set `N_exponential=10` and rerun the program). Does that give something Gaussian? What about 1000?
    Then try to add 10 cauchy numbers (i.e. set `N_cauchy=10` and rerun the program). Does that give something Gaussian? What about 1000?
    If not Gaussian, why do the Cauchy distribution "ruin" the Gaussian distribution?
    And is this in conflict with the Central Limit Theorem?


### Advanced questions:

5. If one used a trunkated mean throwing away the top and bottom e.g. 10%,
    will the truncated mean of 1000 Cauchy numbers then converge to a Gaussian (possibly not with unit width)?

6. What is the lowest number of uniform random numbers needs to be added, before the probability
    for the sum to follow a Gaussian distribution is greater than 1% (on average)
    when considering 1000 sums (i.e. `N_experiments=1000`)? Here, a $\chi^2$ fit is needed.
   

# Learning points:

The exercises should make you:
1. Understand why the Gaussian distribution is central in statistics (since all combinations of distributions converge towards it).
2. Give you a feel for how "fast" this convergence is (i.e. relatively fast for "nice" distributions).
3. Give you confidence in why we consider uncertainties Gaussian, unless known to be otherwise.

Including the Cauchy distribution was simply to make you think about how the Central Limit Theorem is stated!